In [ ]:
!pip install torch transformers accelerate bitsandbytes unsloth

In [ ]:
# Cargar modelo y tokenizador
from transformers import AutoModel, AutoTokenizer
from unsloth import FastLanguageModel
import torch

model_name = "AdrianML7/asesor-finanzas-phi3"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name, #Repo publico
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)

FastLanguageModel.for_inference(model)


In [ ]:
#Nuestro dataset tiene instruccion, input y output
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request. The output must have a clean structure
separated by paragraphs.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

instruction = input("Cual es tu pregunta? ")
input_context = input("Cual es tu perfil? ")

In [ ]:
#Preparar prompt

#Adapta el prompt y tokeniza
prompt = alpaca_prompt.format(instruction, input_context, "")
inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

In [ ]:
#Prepara la respuesta y muestrala por pantalla
from IPython.display import Markdown, display

outputs = model.generate(**inputs, max_new_tokens = 1024, use_cache = True)
texto_crudo = tokenizer.batch_decode(outputs)[0]

# Limpiamos para quedarnos solo con la respuesta del asistente
respuesta_limpia = texto_crudo.split("### Response:\n")[-1].replace("<|endoftext|>", "")

# Renderizar Markdown
display(Markdown(respuesta_limpia))